In [22]:
"""
This is a skeleton of Assignment 3 for DS5110/CS5501 Spring 2024 (UVA).
NOTE you will need to change the .ipynb file name by following the naming convention.
Code should be commented well. 
Feel free to import any missing packages or add extra cells as you see fit. 
"""
from collections import Counter
import socket
import time
import ray
import requests
import os

In [ ]:
#TODO: fill out the public IP address of your Ray head node (vm1).
addr = "ray://<Public_IP_of_VM1>:10001"
ray.init(address=addr)

In [27]:
# You should have a mr-txt folder after you run the script. 
# Go ahead read the files into list
corpus = []
for file_name in os.listdir('mr-txt'):
    file_path = os.path.join('mr-txt', file_name)
    f_txt = open(file_path,'r')
    corpus.extend(f_txt.read().split())

In [ ]:
# You should have a mr-txt folder after you run the script. 
# Go ahead read the files into list
num_partitions = 3
chunk = len(corpus) // num_partitions
partitions = [
    corpus[i * chunk: (i + 1) * chunk] for i in range(num_partitions)
]

In [ ]:
#Task 1: implement Map function
def map_function(document):
    """
    Converts a document into a list of key-value pairs, where each word is a key, and the value is set to 1.
    Steps:
        1. Create an empty list named kvs to store key-value pairs.
        2. Iterate through each word in the document and create a tuple (word, 1) and append it to the kvs list.
        3. After processing all words, return the kvs list as the result.

    Parameters:
    - document (list): List of words representing a document.

    Returns:
    - list: A list of key-value pairs generated from the input document.
    """

    #TODO: start your implementation below this line

    return kvs

In [ ]:
#Task 2: implement Map worker
@ray.remote(scheduling_strategy="SPREAD")
def do_map(corpus, num_partitions=3):
    """
    Applies a mapping operation on the given 'corpus' to partition it into 'num_partitions'.
    Steps:
    1. initialize an empty list named map_results with a length equal to the num_partitions.
    2. Call the map_function on the corpus to generate a list of key-value pairs.
    3. iterate through each entry in key-value pair list and calculate the partition 
        index using the hash(key) the modulo operation with num_partitions to decide which reducer this key-value pair should go to.
    4. return the list of partitions. 

    Parameters:
    - corpus (list): The input data to be mapped.
    - num_partitions (int): The number of partitions to divide the 'corpus' into (default is 3).

    Returns:
    - list: A list containing the mapped partitions of the 'corpus'.
    """

    #TODO: start your implementation below this line
        
    return map_results

In [ ]:
#Task 3: implement reduce function
def reduce_function(key, kvs):
    # TODO: Start your implementation below this line
    return len(kvs[key])

In [ ]:
#Task 4: implement Reduce worker
@ray.remote(scheduling_strategy="SPREAD")
def do_reduce(*my_partition_list):
    """
    Performs a reduce operation on a list of key-value pair partitions.

    Parameters:
    - *my_partition_list: Variable-length list of partitions containing key-value pairs.

    Returns:
    - Any: The result of the reduce operation.
    """
    
    #TODO: start your implementation below this line
                
    return res

In [ ]:
#Task 5: Call this cell after you finish previous tasks.
map_results = [
    do_map.options(num_returns=num_partitions)
    .remote(data, num_partitions)
    for data in partitions
]

In [ ]:
#Task 6: Call this cell after you finish previous tasks.
merged_res = []
for i in range(num_partitions):
    merged_res.append(
        do_reduce.remote(*[partition[i] for partition in map_results])
    )

In [ ]:
#Task 5: Call this cell after you finish previous tasks.
counts = dict()
for res in ray.get(merged_res):
    for key, val in res.items():
        counts[key] =val
        
counts_descending = sorted(counts.items(), key=lambda item: item[1], reverse=True)

In [ ]:
print(counts_descending[:100])

In [ ]:
#Diconnect Ray cluster.
ray.shutdown()